In [ ]:
import yaml
import glob 

all_list = glob.glob("./vendor/*.yaml")

all_list
for file in all_list:
    print(file)

In [ ]:
import requests
from datetime import datetime
#import csv 
import time
import polars as pl
import json

In [ ]:
with open(all_list[0], "r") as vendor_file:
   a = yaml.safe_load(vendor_file)

In [ ]:
class T:
    @staticmethod
    def load_vendor_data(file_path: str) -> dict:
        with open(file_path, "r") as vendor_file:
            print(yaml.safe_load(vendor_file))

In [ ]:
def load_vendor_data(file_path: str) -> dict:
        with open(file_path, "r") as vendor_file:
            print(yaml.safe_load(vendor_file))

In [ ]:
T.load_vendor_data("./vendor/tire-guru.yaml")

In [ ]:
db_inserts_list = [a for file in all_list load_vendor_data(file)]
b = [T.load_vendor_data(file) for file in all_list]
b = list(map(lambda x: x, load_vendor_data()))

In [ ]:
file_path = './Store_20220913_mockfile.csv'

In [ ]:
def zip_fill(zip_code: str) -> str:
    return str(zip_code).zfill(5)

In [ ]:
'''
#dtypes = {col_df:pl.Utf8 for col_df in store_df.columns}
#utf8 to fill null -> need to look at import as more difficult to save string vs cat datatype

Fill by value
df.select(
    [
        pl.exclude('Age'),
        pl.col('Age').fill_null(fill_value=0)
    ]
)'''

In [ ]:
dtypes = {
'Company_ID':pl.Utf8,
 'Store_ID':pl.Utf8,
 'Company_Name':pl.Utf8,
 'Company_Contact_First_Name':pl.Utf8,
 'Company_Contact_Last_Name':pl.Utf8,
 'Company_Contact_Email':pl.Utf8,
 'Company_Contact_Phone_Number':pl.Utf8,
 'Company_Contact_Mobile_Number':pl.Utf8,
 'Company_Address1':pl.Utf8,
 'Company_Address2':pl.Utf8,
 'Company_Address3':pl.Utf8,
 'Company_City':pl.Utf8,
 'Company_State':pl.Utf8,
 'Company_PostalCode':pl.Utf8,
 'Company_Country_Code':pl.Utf8,
 'Store_type':pl.Utf8,
 'Store_Name':pl.Utf8,
 'Store_Contact':pl.Utf8,
 'Store_Address1':pl.Utf8,
 'Store_Address2':pl.Utf8,
 'Store_Address3':pl.Utf8,
 'Store_City':pl.Utf8,
 'Store_State':pl.Utf8,
 'Store_PostalCode':pl.Utf8,
 'Store_Country_Code':pl.Utf8,
 'Number_of_Locations':pl.Utf8,
 'Store_API_Key':pl.Utf8,
 'Saas_Agreement_Signed_Date':pl.Utf8,
 'Signed_By':pl.Utf8,
 }

In [ ]:
store_df = pl.read_csv(file_path, dtypes=dtypes)
#.fill_null("DEFAULT")
store_df2 = store_df.select([pl.all().cast(pl.Categorical)])
#read_csv.lazy for non local storage
#Dict[str,str] to map default values

In [ ]:
store_df.get_columns(["Company_ID", "Signed_By"])
store_df.rows(by_predicate=(pl.col("Signed_By").is_nan()))


In [ ]:
store_3 = store_df.filter(pl.col("Signed_By").is_null())
store_3

In [ ]:
store_3 = store_df2.filter(store_df2.select(pl.all().exclude(["Company_ID", "Signed_By"])) == "DEFAULT")
store_3.collect()

In [ ]:
store_df2.select(pl.col("Saas_Agreement_Signed_Date")).is_null()
good_store = store_df.filter()

In [ ]:
company_group = [
    'Company_ID',
    'Company_Name',
    'Company_Contact_First_Name',
    'Company_Contact_Last_Name',
    'Company_Contact_Email',
    'Company_Contact_Phone_Number',
    'Company_Contact_Mobile_Number',
    'Company_Address1',
    # 'Company_Address2',
    # 'Company_Address3',
    'Company_City',
    'Company_State',
    'Company_PostalCode',
    'Company_Country_Code',
    'Number_of_Locations',
    'Signed_By',
    'Saas_Agreement_Signed_Date'
]

store_group = [
'Store_ID'
'Store_type',
 'Store_Name',
 'Store_Contact',
 'Store_Address1',
 'Store_Address2',
 'Store_Address3',
 'Store_City',
 'Store_State',
 'Store_PostalCode',
 'Store_Country_Code',
#  'Number_of_Locations',
 'Store_API_Key',
]


In [ ]:
lead_df = (store_df2
           .lazy()
           .groupby(company_group)
           .agg(
                [(pl.col('Store_ID')),
                 (pl.col('Store_ID').count()).alias('Store_Count')
                ]
           )
        ).collect()
#lead_df.collect()

In [ ]:
lead_df.filter(pl.col('Number_of_Locations')== 'DEFAULT')\
       .replace('Number_of_Locations', lead_df.select(pl.col('Store_Count')).to_series())

In [ ]:
lead_df.select(pl.col('Company_PostalCode').first())
lead_df.select(
    [
        pl.exclude('Company_PostalCode'),
        pl.col('Company_PostalCode').apply(zip_fill)
    ]
)

In [ ]:
store_df.select([pl.col("Company_Contact_First_Name").sort().head(2)])